Thoughts and notes:

The output will be dark matter relic density, which is best understood on a log scale. Hence the loss function should be mean_squared_logarithmic_error rather than regular mean squared error (mse).

Should split data set into about 70% training, 15% validation, and 15% test data.

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

df = pd.read_csv("result_df.csv")

In [ ]:
def drop_zero_columns(df):
    # Removes all columns from a DataFrame where every value is zero.
    
    # Get columns where all values are zero
    zero_cols = df.columns[(df == 0).all(axis=0)]
    
    # Drop those columns and return
    return zero_cols, df.drop(columns=zero_cols)

In [34]:
col_names, dropped_df = drop_zero_columns(df)
col_names.values

array(['BF_A_to_W_W', 'BF_H_to_Z_h', 'BF_Hp_to_c_b', 'BF_Hp_to_c_s',
       'BF_Hp_to_d_b', 'BF_Hp_to_d_s', 'BF_Hp_to_mu_nu_mu',
       'BF_Hp_to_t_b', 'BF_Hp_to_tau_nu_tau', 'BF_b_1_to_chi_10',
       'BF_b_1_to_chi_1p', 'BF_b_1_to_e_L', 'BF_b_1_to_gl',
       'BF_b_1_to_mu_L', 'BF_b_1_to_nu_e_L', 'BF_b_1_to_nu_mu_L',
       'BF_b_1_to_nu_tau_1', 'BF_b_1_to_tau_1', 'BF_b_2_to_chi_10',
       'BF_b_2_to_chi_1p', 'BF_b_2_to_gl', 'BF_chi_10_other',
       'BF_chi_10_to_e_L', 'BF_chi_10_to_e_R', 'BF_chi_10_to_gravitino_Z',
       'BF_chi_10_to_gravitino_gam', 'BF_chi_10_to_gravitino_h',
       'BF_chi_10_to_mu_L', 'BF_chi_10_to_mu_R', 'BF_chi_10_to_nu_e_L',
       'BF_chi_10_to_nu_mu_L', 'BF_chi_10_to_nu_tau_1',
       'BF_chi_10_to_tau_1', 'BF_chi_10_to_tau_2', 'BF_chi_1p_to_chi_30',
       'BF_chi_20_to_chi_1p_ele_nu_ele', 'BF_chi_20_to_chi_1p_mu_nu_mu',
       'BF_chi_20_to_chi_1p_tau_nu_tau', 'BF_chi_20_to_chi_2p',
       'BF_chi_20_to_chi_2p_W', 'BF_chi_30_to_chi_2p',
       'BF_chi_

In [26]:
data = {"A": [0,0,0],
        "B": [0,1,2],
        "C": [0,0,1]}

test = pd.DataFrame(data=data)
col_names, dropped = drop_zero_columns(test)

print(col_names.values)

['A']


In [23]:
dropped.head()

,B,C
0,0,0
1,1,0
2,2,1
